SNAFU to decimal, decimal to SNAFU conversions

# SNAFU to decimal

In [8]:
def snafu_to_decimal(snafu_str):
    s=snafu_str.strip()
    N=len(s)

    value=0

    for i,ch in enumerate(s):
        power5 = 5**(N-i-1)
        
        if ch=='2':
            value+=2*power5
            continue
        if ch=='1':
            value+=power5
            continue
        if ch=='0':
            #Don't add anything
            continue
        if ch=='=':
            value-=2*power5
            continue
        if ch=='-':
            value-=power5
        else:
            raise ValueError(f"{ch} is not a valid SNAFU character")
    
    return value

In [9]:
#test, expect 3
snafu_to_decimal('1=')

3

In [10]:
#test, expect 1747
snafu_to_decimal('1=-0-2')

1747

In [117]:
#test, expect 1257
snafu_to_decimal('20012')

1257

# Decimal to SNAFU (not working)

Conversion should use log base 5

In [25]:
import math

In [26]:
math.log(125,5)

3.0000000000000004

In [27]:
math.log(1.0,5)

0.0

In [28]:
math.log(3.0,5)

0.6826061944859854

In [33]:
for i in range(1,15):
    v=math.log(float(i),5.0)
    print(f"{i}: {v}")

1: 0.0
2: 0.43067655807339306
3: 0.6826061944859854
4: 0.8613531161467861
5: 1.0
6: 1.1132827525593785
7: 1.2090619551221675
8: 1.2920296742201791
9: 1.3652123889719707
10: 1.4306765580733933
11: 1.4898961024049782
12: 1.5439593106327716
13: 1.5936926411670824
14: 1.6397385131955606


transition of numbering is 1->3 and 7->8

In [70]:
#Test value here
v=8.0

In [71]:
pow5=math.log(v, 5.0)
pow5

1.2920296742201791

In [72]:
pow5_int = math.floor(pow5)
pow5_1 = pow5_int+1
pow5_1

2

In [74]:
v_pow5_1 = 5.0**(pow5_1)
v/v_pow5_1

0.32

In [73]:
v_pow5_1 = 5.0**(pow5_1)
print(v_pow5_1)
v_pow5 = 5.0**(pow5_int)
print(v_pow5)
diff = v_pow5_1-v
print(diff)

25.0
5.0
17.0


In [75]:
# From https://stackoverflow.com/questions/2267362/how-to-convert-an-integer-to-a-string-in-any-base
def numberToBase(n, b):
    if n == 0:
        return [0]
    digits = []
    while n:
        digits.append(int(n % b))
        n //= b
    return digits[::-1]

In [77]:
for i in range(1,15):
    print(f"{i}: base5:{numberToBase(i,5)}")

1: base5:[1]
2: base5:[2]
3: base5:[3]
4: base5:[4]
5: base5:[1, 0]
6: base5:[1, 1]
7: base5:[1, 2]
8: base5:[1, 3]
9: base5:[1, 4]
10: base5:[2, 0]
11: base5:[2, 1]
12: base5:[2, 2]
13: base5:[2, 3]
14: base5:[2, 4]


Maybe can use conversion to base 5, and then convert to SNAFU

In [110]:
#test value
v=314159265

In [111]:
dig = numberToBase(v,5)
print(dig)

[1, 1, 2, 0, 4, 1, 1, 0, 4, 4, 0, 3, 0]


In [112]:
s_snafu=""
carry=0
for i in range(len(dig)):
    d0 = dig[-(i+1)]
    print(d0)

    v0 = d0+carry

    if v0==3:
        s_snafu+="="
        carry=1
        continue
    if v0==4:
        s_snafu+="-"
        carry=1
        continue
    if v0==5:
        s_snafu+="0"
        carry=1
        continue
    #else
    s_snafu+=str(v0)
    carry=0

if carry>0:
    s_snafu+=str(carry)
print(s_snafu)
#At the end we need to invert the string
s_snafu_invert = s_snafu[::-1]
print(s_snafu_invert)


0
3
0
4
4
0
1
1
4
0
2
1
1
0=1-0111-1211
1121-1110-1=0


OK

In [113]:
def decimal_to_snafu(v):
    assert v>=1.0, "Error value is not >=1.0"

    dig = numberToBase(v,5)

    s_snafu=""
    carry=0
    for i in range(len(dig)):
        d0 = dig[-(i+1)]

        v0 = d0+carry
        if v0==3:
            s_snafu+="="
            carry=1
            continue
        if v0==4:
            s_snafu+="-"
            carry=1
            continue
        if v0==5:
            s_snafu+="0"
            carry=1
            continue
        #else
        s_snafu+=str(v0)
        carry=0

    if carry>0:
        s_snafu+=str(carry)
    #print(s_snafu)
    #At the end we need to invert the string
    s_snafu_invert = s_snafu[::-1]
    #print(s_snafu_invert)

    return s_snafu_invert


In [115]:
for i in range(1,21):
    print(f"{i}, {decimal_to_snafu(i)}")

1, 1
2, 2
3, 1=
4, 1-
5, 10
6, 11
7, 12
8, 2=
9, 2-
10, 20
11, 21
12, 22
13, 1==
14, 1=-
15, 1=0
16, 1=1
17, 1=2
18, 1-=
19, 1--
20, 1-0


In [116]:
decimal_to_snafu(314159265)

'1121-1110-1=0'

OK